# **1. Image Segmentation**
* 컴퓨터 비전 분야에서 이미지나 비디오의 디지털 데이터를 여러 개의 부분 또는 객체로 분할하는 기술
* 이미지의 중요한 요소들을 식별하고 각 요소를 개별적으로 분석할 수 있게 하는 것

# **2. Image Segmentation의 유형**
* Semantic Segmentation
    * 이미지의 각 픽셀을 미리 정의된 클래스 레이블 중 하나로 분류
    * 예) 도로, 차선, 보행자 등을 식별
* Instance Segmentation
    * 동일한 클래스 내의 서로 다른 개체들을 개별적으로 식별
    * 예) 이미지 내의 개별 물체 수를 파악하고 각각 물체를 식별 및 추적하는 경우
* Panoptic Segmentation
    * Semantic Segmentation, Instance Segmentation을 결합한 형태
    * 배경과 같은 클래스를 처리하는 Semantic Segmentation과, 객체를 구분하는 Instance Segmentation을 모두 수행
    * 예) 풍경 이미지에서 하늘, 도로, 나무와 사람, 자동차를 동시에 식별

# **3. 차량 파손 검사**

In [ ]:
import os
import random
import shutil
import cv2
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
data_root = '/content/drive/MyDrive/컴퓨터비전 시즌2/6. 프로젝트/4. Segmentation'
file_root = f'{data_root}/data'
cls_list = ['Scratched', 'Separated', 'Crushed', 'Breakage']
project_name = 'cd'

In [ ]:
train_root = f'{data_root}/{project_name}/train'
valid_root = f'{data_root}/{project_name}/valid'
test_root = f'{data_root}/{project_name}/test'

for folder in [train_root, valid_root, test_root]:
    if not os.path.exists(folder):
        os.makedirs(folder)
    for s in ['images', 'labels']:
        s_folder = f'{folder}/{s}'
        if not os.path.exists(s_folder):
            os.makedirs(s_folder)

In [ ]:
# 라벨링 형태 변환(YOLO): [cls xc, yc, w, h]
# [[[x1, y1], [x2, y2], ...]] -> normalize
def json_to_yolo_polygon(polygon, w, h):
    yolo_list = []
    for p in polygon:
        yolo_list.append(p[0]/w)
        yolo_list.append(p[1]/h)
    return ' '.join([str(x) for x in yolo_list])

In [ ]:
file_list = glob.glob(f'{file_root}/annotations/*.json')
random.seed(2024)
random.shuffle(file_list)
print(len(file_list))

1200


In [ ]:
# 라벨링 형태 변환 파일 생성(json -> txt)
# 테스트해보기
mask_name = file_list[0].split('/')[-1].replace('json', 'txt')
result = []

with open(file_list[0], 'r') as json_file:
    data = json.load(json_file)
    h = data['images']['height']
    w = data['images']['width']
    for ann in data['annotations']:
        label = ann['damage']
        if label in cls_list:
            polygon_coord = ann['segmentation'][0][0][:-1]
            cood_string = json_to_yolo_polygon(polygon_coord, w, h)
            yolo_string = f'{cls_list.index(label)} {cood_string}'
            result.append(yolo_string)

In [ ]:
result

['0 0.246875 0.5876591576885406 0.278125 0.5866797257590598 0.2859375 0.6082272282076395 0.25625 0.6190009794319294 0.24375 0.614103819784525 0.246875 0.6003917727717923',
 '0 0.6796875 0.41136141038197843 0.69140625 0.40450538687561216 0.69453125 0.40646425073457393 0.6859375 0.4201762977473066 0.6796875 0.42213516160626835 0.6796875 0.4152791380999021']

In [ ]:
# 라벨링 형태 변환 파일 생성(json -> txt)
if not os.path.isdir(f'{file_root}/labels'):
    os.mkdir(f'{file_root}/labels')

for file in tqdm(file_list):
    result = []
    with open(file, 'r') as json_file:
        data = json.load(json_file)
        h = data['images']['height']
        w = data['images']['width']
        for ann in data['annotations']:
            label = ann['damage']
            if label in cls_list:
                polygon_coord = ann['segmentation'][0][0][:-1]
                cood_string = json_to_yolo_polygon(polygon_coord, w, h)
                yolo_string = f'{cls_list.index(label)} {cood_string}'
                result.append(yolo_string)
    if result:
        save_path = file.replace('annotations', 'labels').replace('json', 'txt')
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(result))

100%|██████████| 1200/1200 [19:04<00:00,  1.05it/s]


In [ ]:
file_list = glob.glob(f'{file_root}/labels/*.txt')
random.shuffle(file_list)
test_ratio = 0.1
num_file = len(file_list)

test_list = file_list[:int(num_file*test_ratio)]
valid_list = file_list[int(num_file*test_ratio):int(num_file*test_ratio)*2]
train_list = file_list[int(num_file*test_ratio)*2:]

for i in test_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{test_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'{file_root}/images/{img_name}'
    shutil.copyfile(img_path, f'{test_root}/images/{img_name}')

for i in valid_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{valid_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'{file_root}/images/{img_name}'
    shutil.copyfile(img_path, f'{valid_root}/images/{img_name}')

for i in train_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{train_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'{file_root}/images/{img_name}'
    shutil.copyfile(img_path, f'{train_root}/images/{img_name}')

In [ ]:
project_root = '/content/drive/MyDrive/컴퓨터비전 시즌2/6. 프로젝트/4. Segmentation'

In [ ]:
%cd /content/drive/MyDrive/컴퓨터비전 시즌2/6. 프로젝트/4. Segmentation

In [ ]:
!pip install ultralytics

In [ ]:
import yaml
import ultralytics
from ultralytics import YOLO

In [ ]:
ultralytics.checks()

In [ ]:
# yaml 파일만들기
data = dict()
data['train'] = train_root
data['val'] = valid_root
data['test'] = test_root
data['nc'] = len(cls_list)
data['names'] = cls_list

with open(f'{project_root}/car_damage.yaml', 'w') as f:
    yaml.dump(data, f)

In [ ]:
%cd /content/drive/MyDrive/컴퓨터비전 시즌2/6. 프로젝트/4. Segmentation

In [ ]:
# YOLO 객체 만들기
# yolov8s-seg.yaml: 모델의 구조와 설정을 정의하는 파일. 처음부터 학습
# model = YOLO('yolov8s-seg.yaml')
# yolov8s-seg.pt: 학습된 모델을 사용하여 바로 세그멘테이션 작업을 할 때
model = YOLO('yolov8s-seg.pt')

In [ ]:
# 학습시키기
results = model.train(data='car_damage.yaml', epochs=2, batch=16, imgsz=224, device=0, workers=4, amp=False, patience=30, name='yolo_s')

In [ ]:
%cd /content/drive/MyDrive/컴퓨터비전 시즌2/6. 프로젝트/4. Segmentation

In [ ]:
# best.pt로 YOLO 객체 만들기
project_root = '/content/drive/MyDrive/컴퓨터비전 시즌2/6. 프로젝트/4. Segmentation'
result_folder = f'{project_root}/runs/segment'

In [ ]:
model = YOLO(f'{result_folder}/yolo_s/weights/best.pt')

In [ ]:
# test 데이터로 val 하기
metrics = model.val(split='test')

In [ ]:
%cd /content/drive/MyDrive/컴퓨터비전 시즌2/6. 프로젝트/4. Segmentation

In [ ]:
# test 데이터 모두 불러와 셔플하기
test_file_list = glob.glob(f'{test_root}/images/*')
random.shuffle(test_file_list)

In [ ]:
# 그 중 첫번쨰 파일을 모델에 넣고 예측하기
test_img = cv2.imread(test_file_list[0])
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
result = model(img_src)[0]

In [ ]:
# polygon2mask: 이미지에서 다각형 형태의 영역을 마스크로 변환하는 함수로
# 주로 이미지 처리와 컴퓨터 비전 작업에서 사용
from skimage.draw import polygon2mask

In [ ]:
image_shape = (100, 100)
polygon = np.array([[10, 10], [80, 20], [50, 80]])
mask = polygon2mask(image_shape, polygon)
plt.imshow(mask, cmap='gray') # mask: 자신이 관심있는 영상만 딱 흑백으로 만들어주는 기법
plt.show()

In [ ]:
test_img = cv2.imread(test_file_list[1])
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
result = model(img_src)[0]

result_mask = np.zeros(test_img.shape[:2])
masks = result.masks
for m in masks:
    polygon_coor = m.xy[0]
    # 주어진 이미지 크기와 폴리곤 좌표를 사용하여 해당 영역을 1로 채운 마스크를 생성
    # 나머지는 0으로 유지
    mask = polygon2mask(test_img.shape[:2], polygon_coor)
    # maximum(): 두 배열의 요소별 최대값을 반환하므로 여러 객체의 마스크가 겹치더라도
    # 최대값을 유지
    result_mask = np.maximum(mask, result_mask)
    # 2D 마스크를 3D 배열로 변환하고 repeat() 사용해 동일한 값을 3개의 채널에 복사
result_mask = np.repeat(result_mask[:, :, np.newaxis], 3, -1)

plt.subplot(1, 2, 1)
plt.imshow(img_src)
plt.subplot(1, 2, 2)
plt.imshow(result_mask)
plt.show()